### Import Statements

In [ ]:
# Import Statements
import json
import requests
import pandas as pd
import numpy as np
import os

FILENAME = "AddressesTest"

# Open API Key
with open("Keys.json", "r") as data:
    Keys = json.load(data)

### Known Errors
1. Subtour elimination constraint: Work in progress, currently returns an infeasible solve.
            - However, the model will solve without it, but does return a route with subtours

### Next Up In Development
1. Finish subtour elimination constraint
    - Matter of finishing figuring out how to write it, variable generation should be done
2. .sol file interpretation
    - This is assuming that the LP files get manually uploaded to NEOS for solving
    - CPLEX there can return a .sol file that we can parse and interpret solutions from
3. Data display
    - A project is no good if you can't understand the results
    - Still need to find a good way of showing the solution on a graph
    - Could be a quick and dirty NetworkX thing, or a more sophisticated MapBox solution
4. If time, remote solve
    - Currently make LP file, then upload it manually to solve
    - Would be nice to instead change it into an xml approach that can be sent to NEOS via script
    - Removes time and human error, also allows for everything to be done in one script, not two

### Creating the Distance Matrix

In [ ]:
# Define API Call Function
def RouteCaller(loc1:str, loc2:str,  key:str=None, units:str='meters', routeType:str='BICYCLE') -> float:
    """ Function that makes a single API call between two distances

    # Arguments #
    :arg loc1: Starting location for the route
    :type loc1: str
    :arg loc2: Ending location for the route
    :type loc2: str
    :arg units: Measuring system for the route, metric of imperial
    :type units: str
    :arg routeType: Mode of travel for the route. Default is Bicyle because that's the purpose of this project
    :type routeType: str
    """
    print(f'Getting data for {loc1} to {loc2}')
    # Verify Key exists
    if key == None:
        raise ValueError('No API Key has been passed. Please insert key and try again')
    # Ping the API
    retVal = requests.get(
        f"https://maps.googleapis.com/maps/api/directions/json?destination={loc1}&origin={loc2}&units={units}&mode={routeType}&key={key}").json()
    if 'error_message' in retVal.keys():
        # Error handling
        print(
            f"Error when pinging API, issue: {retVal['error_message']}")
        return np.NaN
    else:
        # Return the route information
        return float(retVal['routes'][0]['legs'][0]['distance']['text'][:-3])

# Define the Distance Matrix
def generateDistanceMatrix(df:pd.DataFrame, key:str) -> np.ndarray:
    """ Takes in a DataFrame, and returns a numpy array

    # Arguments #
    :arg df: all of the addresses in the problem, placed in a pandas DataFrame
    :type df: pd.DataFrame
    :arg key: API key, so that Google knows who is calling
    :type key: str

    # Returns #
    :return distMatrix: A matrix of size NxN, where N is the number of locations in DataFrame df
    :rtype distMatrix: np.ndarray
    """
    # Local array storage
    distDict = {}
    arrays = []

    # Iterate through the DataFrame
    for index1, row1 in df.iterrows():
        distDict[index1] = []
        for index2, row2 in df.iterrows():
            if index1 == index2:
                # If location equals itself then give 0 for distance, no need to ping the API
                distDict[index1].append(0)

            elif index1 != index2:
                # Ping Google's Destination API for route data
                distDict[index1].append(RouteCaller(df.iloc[index1]['Address'], df.iloc[index2]['Address'], key))

    # Process data into arrays, stack, and return
    for key in distDict.keys():
        arrays.append(np.array(distDict[key]))
    distMatrix = np.vstack(arrays)
    return distMatrix


def validateCache(filename:str,df:pd.DataFrame,key:str=None) -> np.ndarray:
    """ Validate and load cached data, to prevent unnecessary API calls

    # Arguments #
    :arg filename: ending of filename to check if it exits. If it doesn't we will end up making it
    :type filename: str
    :arg df: If no cache, need data to pass to route generator
    :type df: pd.DataFrame
    :arg key: If no cache, need key for API call
    :type key: str

    # Returns #
    :return distMatrix: Returns a distmatrix
    :rtype distMatrix: np.ndarray
    """
    # Define where the cache should be
    cache_filename = os.path.splitext(filename)[0] + ".npy"
    # Check for if the parent folder exists, if not make one
    if not os.path.exists(os.path.join(os.path.dirname(__file__),"CachedDistances")):
        os.mkdir(os.path.join(os.path.dirname(__file__),"CachedDistances"))
    # Check if the distance matrix doesn't exist, if so make one
    if not os.path.exists(os.path.join(os.path.dirname(__file__),"CachedDistances",cache_filename)):
        distMatrix = generateDistanceMatrix(df, key)
        np.save(os.path.join(os.path.dirname(__file__),"CachedDistances",cache_filename),distMatrix)
        return distMatrix
    # Since the matrix exists, just load it
    else:
        distMatrix = np.load(os.path.join(os.path.dirname(__file__), "CachedDistances", cache_filename))
        return distMatrix

### Define Constraints

In [ ]:
def generateContraintMatrix(distMatrix:np.ndarray) -> np.ndarray:
    """ Generate the constraint matrix from given distance matrix

    # Arguments #
    :arg distMatrix: Big distance matrix that we calculated in the last step
    :type distMatrix: np.ndarray

    # Returns #
    :return retVal: Combined constraint matrix, with top and bottom halves stacked nicely
    :rtype retVal: np.ndarray
    """
    NumElements = len(distMatrix)
    """
        Generates the top portion of the constraint matrix, a 1 x N matrix of ones for index i in N. Here is an example with N = 3.
        [ 0 0 0 0 0 0 0 0 0 ]        [ 1 1 1 0 0 0 0 0 0 ]
        | 0 0 0 0 0 0 0 0 0 | --->   | 0 0 0 1 1 1 0 0 0 |
        [ 0 0 0 0 0 0 0 0 0 ]        [ 0 0 0 0 0 0 1 1 1 ]
    """
    retArrayTop = np.zeros(shape=(NumElements,NumElements**2))
    for index in range(NumElements):
        retArrayTop[index,index*NumElements:(index+1)*NumElements] = np.ones(shape=(1,NumElements))
    """
        Generates the bottom portion of the constraint matrix, a N x N**2 matrix full of horizontally aligned identity N x N matricies.
        Example with N = 3.
        [ 1 0 0 ]   [ 1 0 0 ]   [ 1 0 0 ]         [ 1 0 0 1 0 0 1 0 0 ]
        | 0 1 0 | + | 0 1 0 | + | 0 1 0 |   --->  | 0 1 0 0 1 0 0 1 0 |
        [ 0 0 1 ]   [ 0 0 1 ]   [ 0 0 1 ]         [ 0 0 1 0 0 1 0 0 1 ]
    """
    individualBottoms = tuple(np.identity(NumElements) for i in range(NumElements))
    retArrayBottom = np.hstack(individualBottoms)
    retVal = np.vstack((retArrayTop,retArrayBottom))
    #retVal = np.vstack((retVal,np.ones(shape=(1,NumElements**2),dtype=np.uint8)))
    return retVal

### Write to LP File
This is the most jank part, and needs optimization and clean up

In [ ]:
def lpGenerator(distMatrix:np.ndarray, constraintMatrix:np.ndarray,filename:str) -> np.ndarray:
    """ Writes the lp file for the constraint matrix. LP Files are the way we give the solver our problem.

    # Arguments #
    :arg distMatrix: a distance matrix of size N x N. This contains the necessary objective information
    :type distMatrix: np.ndarray
    :arg constraintMatrix: the constraint matrix with binary variables
    :type constraintMatrix: np.ndarray
    :arg filename: input filename that we will make the associated lp file for
    :type filename: str
    """
    
    # Define where the LP file should be
    lp_filename = os.path.splitext(filename)[0] + ".lp"
    # Check for if the parent folder exists, if not make one
    if not os.path.exists(os.path.join(os.path.dirname(__file__),"LPFiles")):
        os.mkdir(os.path.join(os.path.dirname(__file__),"LPFiles"))
    full_lp_filename = os.path.join(os.path.dirname(__file__),"LPFiles",lp_filename)
    # Delete old lp file of same name
    if os.path.exists(full_lp_filename):
        os.remove(full_lp_filename)

    # Find N
    NumElements = len(distMatrix)

    # Variable Creation
    vars_dict = {}
    for i in range(NumElements):
        vars_dict[i] = {}
        for j in range(NumElements):
            vars_dict[i][j] = f'X{i}Y{j}'
    subtour = [f'U{i}' for i in range(NumElements)]
    """
    An LP File stands for Linear Programming File. This is the input that solvers like CPLEX prefer to take in.
    LP Files are organized into logical sections. Each section is started by a keyword.
    The first section is the objective, and has keywords: max, maximum, maximize, min, minimum, minimize.
    """
    # Write out cost * variable, put in a list
    costs = []
    for i in vars_dict.keys():
        for j in vars_dict[i].keys():
            cost = distMatrix[i, j]
            if cost != 0.0:
                costs.append(f'{cost} {vars_dict[i][j]}')
                costs.append('+')
    costs.pop()
    print(costs)
    '''
    The second section is the constraint section. This is where we tell the solver the rules that it must
    follow in order to obtain a feasible solution. We start this section using "subject to"
    '''
    lp_constraints = []
    # Top half
    loc = 0
    for i in vars_dict.keys():
        local = []
        for j in vars_dict[i].keys():
            if i != j:
                if constraintMatrix[i,j+loc] == 1.0:
                    local.append(vars_dict[i][j])
                    local.append(' + ')
        local.pop()
        lp_constraints.append(local)
        loc += NumElements
    # Bottom Half
    loc = 0
    for i in vars_dict.keys():
        local = []
        for j in vars_dict[i].keys():
            if i != j:
                if constraintMatrix[i+NumElements,j*NumElements+loc] == 1.0:
                    local.append(vars_dict[j][i])
                    local.append(' + ')
        local.pop()
        lp_constraints.append(local)
        loc += 1

    # Write the LP file
    with open(full_lp_filename, 'x') as lp:
        # Objective Section
        lp.write('Min \n')
        for eq in costs:
            lp.write(f'{eq} ')
        # Constraint Section
        lp.write('; \nsubject to \n')
        for eq in lp_constraints:
            for i in eq:
                lp.write(i)
            lp.write(' = 1 \n')
        # Subtour elimination
        for i in vars_dict.keys():
            for j in vars_dict.keys():
                lp.write(f'{subtour[j]} - {subtour[i]} - {vars_dict[i][j]} >= {2*NumElements-1} \n')
        # Define Variable Types
        lp.write('bin \n')
        for i in vars_dict.keys():
            for j in vars_dict[i].keys():
                if i != j:
                    lp.write(f'{vars_dict[i][j]} ')
        for i in subtour:
            lp.write(f'{i} ')
        lp.write('\nEND')

### Define main, run file
This has all been done in a main.py file so far, and has not been directly adapted for a jupyter notebook.

In [ ]:
def main(filename:str,key:str=None):
    """ We put all the work inside this one function, so that at the end of this program we only have to call one function

    # Arguments #
    :arg filename: Generic name of the input data file
    :type filename: str
    :arg key: API key needed to call Google Maps if no cached data
    :type key: str
    """

    # Open the data, put it in Pandas
    with open(f"Data/{FILENAME}.csv", "r") as data:
        df = pd.read_csv(data)
        # Make the addresses API friendly
        df = df.replace(' ', '+', regex=True)
    # Get distance matrix
    distMatrix = validateCache(filename,df,key)
    constMatrix = generateContraintMatrix(distMatrix)
    lpGenerator(distMatrix, constMatrix, filename)

if __name__ == "__main__":
    main(filename=FILENAME, key=Keys['googleMaps'])

Manually call main I guess

In [ ]:
main(filename=FILENAME, key=Keys['googleMaps'])